In [3]:
# Exercise 3

import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial import distance

# Generates an array of 2D points and desired size with random numbers in the specified interval (including the
# low and high value) and save it as a txt file, where each point is separated by a new line and each value by
# a comma. The first line denotes the label of each point value ('x' and 'y'), also separated by a comma.
def generate(low, high, size):
    generator = np.random.default_rng()
    points = generator.integers(low, high, size*2, endpoint=True).reshape((size, 2))
    np.savetxt(filename, points, fmt='%0d', delimiter=',', newline='\n', header='x,y', comments='')
    return points

# Reads the txt file written by generate() and returns a 2D array where each row represents a point with the
# x and y coordinates as columns
def read():
    with open(filename) as f:
        lines = f.readlines()[1:]
        x = [line.split(',')[0] for line in lines]
        y = [line.split(',')[1].rstrip() for line in lines]
        return np.stack((np.array(x, dtype=int), np.array(y, dtype=int)), axis=-1)

# Draws a scatter plot of the clusters returned by the k_means function
def draw_clusters(clusters):
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    i = 0
    for c in clusters:
        centroid, points = c
        color_string = 'C' + str(i)
        ax.scatter(points[:, 0], points[:, 1], marker='o', color=color_string)
        if len(centroid) > 1:
            ax.scatter(centroid[0], centroid[1], marker='x', color=color_string, s=200)
        i = i + 1
    plt.show()

# Convenience function to draw scatter plots of the points in files written by generate() that are not yet clustered
def draw():
    # Read the txt file
    points = read()
    draw_clusters([([], points)])

# Recursive loop used by k_means()
def k_means_loop(points, cluster_centroids, cluster_affiliation):
    # Recover k
    k = cluster_centroids.shape[0]
    # Calculate squared distances of all points from all cluster centroids
    distances = np.empty((points.shape[0], k)).ravel()
    i = 0
    for p in points:
        for c in cluster_centroids:
            distances[i] = distance.cdist([p], [c], 'sqeuclidean')
            i = i + 1
    # Reshape array so that rows represent points and columns clusters
    distances = distances.reshape((points.shape[0], k))
    # Determine closest cluster (Bool array with the same shape as the distances array)
    new_cluster_affiliation = np.zeros(distances.shape, dtype=bool)
    i = 0
    # Iterate through every point
    for d in distances:
        # Set value from False to True in the corresponding row (point) and column (cluster)
        new_cluster_affiliation[i][np.argmin(d)] = True
        i = i + 1
    # Compare old and new cluster affiliation
    if (np.all((new_cluster_affiliation == cluster_affiliation))):
        # No change, exit recursion and return array of clusters (each as a tuple with the centroid
        # followed by an array of the affiliated points) and sum of squared errors
        result = []
        # Index the distances array with the new_cluster_affiliation array and sum up the values
        sse = np.sum(distances[new_cluster_affiliation])
        i = 0
        for c in new_cluster_affiliation.transpose():
            if(np.any(c)):
                result.append((cluster_centroids[i], points[c]))
            i = i+1
        return result, sse
    else:
        # Calculate new cluster centroids
        i = 0
        # Iterate through transposed affiliation array (each row is a cluster, each column a point)
        for c in new_cluster_affiliation.transpose():
            # Skip empty clusters
            if not(np.any(c)): continue
            # Centroid is defined as the mean of the points that belong to the cluster (indexing with boolean array c)
            cluster_centroids[i] = np.array([np.average(points[c][:, 0]), np.average(points[c][:, 1])])
            i = i + 1
        # Next iteration
        return k_means_loop(points, cluster_centroids, new_cluster_affiliation)

# Find k_means clusters of the points in files written by generate() and return them as an array tuples with the
# centroid followed by an array of the affiliated points, as well as the sum of squared errors
def k_means(k):
    # Read the txt file
    points = read()
    # Randomly select initial cluster centers
    generator = np.random.default_rng()
    indices = generator.integers(0, points.shape[0], k, endpoint=False)
    initial_cluster_centroids = np.take(points, indices, axis=0)
    # initial_cluster_centroids = initial_cluster_centroids.astype(np.dtype(float))
    initial_cluster_affiliation = np.zeros((points.shape[0], k), dtype=bool)
    # Go into recursive loop
    return k_means_loop(points, initial_cluster_centroids, initial_cluster_affiliation)

# Tests the spatial randomness of the points in files written by generate()
# Returns Hopkins Statistic: uniform datasets have a value of about 0.5, highly skewed datasets have a value close to 0
def hopkins_statistic(samples):
    # Read the txt file
    points = read()
    # Pick random sample
    rng = np.random.default_rng()
    sample_random = rng.choice(points, samples)
    # Pick uniform sample
    sample_uniform = points[:samples]
    # Loop through both samples and sum up the distances to the nearest neighbour of each sample
    sum_dist_random, sum_dist_uniform = 0, 0
    for i in range(sample_random.shape[0]):
        min_dist_random, min_dist_uniform = np.inf, np.inf
        for j in points:
            dist_random = np.sqrt(np.square(sample_random[i][0]-j[0]) + np.square(sample_random[i][1]-j[1]))
            dist_uniform = np.sqrt(np.square(sample_uniform[i][0]-j[0]) + np.square(sample_uniform[i][1]-j[1]))
            if dist_random > 0 and dist_random < min_dist_random:
                min_dist_random = dist_random
            if dist_uniform > 0 and dist_uniform < min_dist_uniform:
                min_dist_uniform = dist_uniform
        sum_dist_random = sum_dist_random + min_dist_random
        sum_dist_uniform = sum_dist_uniform + min_dist_uniform
    return sum_dist_uniform / (sum_dist_random + sum_dist_uniform)


# Main
filename, min, max, size, N = 'points.txt', 0, 245, 1000, 101

print('\nGenerating {} points with x and y coordinates ranging from {} to {} and saving them to {}'.format(size, min, max, filename))
generate(min, max, size)
draw()
print('\nHopkins Statistic: {}'.format(hopkins_statistic(100)))

print('\nClustering with k-means ranging from 2 to {}'.format(N-1))
sse_array = np.empty(N - 2)
for i in range(2, N):
    result = k_means(i)
    sse_array[i-2] = result[1]
    print('k = {:3d}, SSE = {:7.0f}'.format(i, result[1]))

draw_clusters(k_means(10)[0])
plt.plot(sse_array)
plt.show()




Generating 1000 points with x and y coordinates ranging from 0 to 245 and saving them to data/points.txt


FileNotFoundError: [Errno 2] No such file or directory: 'data/points.txt'